<a href="https://colab.research.google.com/github/once-upon-an-april/Thuc-Hanh-Deep-Learning-trong-Khoa-Hoc-Du-Lieu-DS201.Q11.1/blob/main/Bai1/22520975_Lab2_Bai4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import numpy as np
import os
import time
import copy
from tqdm.auto import tqdm

In [ ]:
# Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [ ]:
# --- Cấu hình chung ---
DATA_DIR = '/content/drive/MyDrive/Data/VinaFood21/VinaFood21'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {device}")

if not os.path.exists(DATA_DIR):
    print(f"LỖI: Không tìm thấy thư mục '{DATA_DIR}'.")
    print("Vui lòng kiểm tra lại đường dẫn Google Drive của bạn.")

NUM_CLASSES = 21 # VinaFood21

Sử dụng thiết bị: cuda:0


In [ ]:
def get_dataloaders(img_size, batch_size=32):
    """Tải dữ liệu với kích thước ảnh cụ thể."""
    print(f"\nĐang tải dữ liệu với kích thước {img_size}x{img_size}...")
    data_transforms = {
        'train': transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'test': transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }
    try:
        image_datasets = {x: datasets.ImageFolder(os.path.join(DATA_DIR, x), data_transforms[x])
                          for x in ['train', 'test']}
        dataloaders = {x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=(x=='train'), num_workers=2)
                       for x in ['train', 'test']}
        dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
        class_names = image_datasets['train'].classes
        print(f"Tải xong. {dataset_sizes['train']} ảnh train, {dataset_sizes['test']} ảnh test.")
        return dataloaders, dataset_sizes, class_names
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy thư mục 'train' hoặc 'test' trong {DATA_DIR}")
        return None, None, None

In [ ]:
def quick_evaluate(model, dataloader):
    """Hàm đánh giá nhanh (chỉ tính Acc) để xem sau mỗi epoch."""
    model.eval()
    running_corrects = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)
            total += labels.size(0)
    acc = running_corrects.double() / total
    print(f'Test Acc (Validation): {acc:.4f}')
    model.train() # Chuyển lại chế độ train

In [ ]:
def train_model_torchvision(model, dataloaders, dataset_sizes, criterion, optimizer, num_epochs):
    """Hàm huấn luyện cho LeNet và ResNet (output chuẩn)."""
    since = time.time()
    model = model.to(device)

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        model.train() # Đặt mô hình ở chế độ training
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in tqdm(dataloaders['train'], desc=f"Training Epoch {epoch+1}"):
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()

            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes['train']
        epoch_acc = running_corrects.double() / dataset_sizes['train']
        print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        # Đánh giá nhanh trên tập test (validation) sau mỗi epoch
        quick_evaluate(model, dataloaders['test'])


    time_elapsed = time.time() - since
    print(f'\nHuấn luyện hoàn tất trong {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    return model

In [ ]:
def train_model_googlenet(model, dataloaders, dataset_sizes, criterion, optimizer, num_epochs):
    """Hàm huấn luyện đặc biệt cho GoogLeNet (có 3 output)."""
    since = time.time()
    model = model.to(device)

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        model.train() # Đặt mô hình ở chế độ training
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in tqdm(dataloaders['train'], desc=f"Training Epoch {epoch+1}"):
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()

            with torch.set_grad_enabled(True):
                main_out, aux1_out, aux2_out = model(inputs)
                loss_main = criterion(main_out, labels)
                loss_aux1 = criterion(aux1_out, labels)
                loss_aux2 = criterion(aux2_out, labels)
                loss = loss_main + 0.3 * (loss_aux1 + loss_aux2)
                _, preds = torch.max(main_out, 1)
                loss.backward()
                optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes['train']
        epoch_acc = running_corrects.double() / dataset_sizes['train']
        print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        # Đánh giá nhanh trên tập test (validation) sau mỗi epoch
        quick_evaluate(model, dataloaders['test'])

    time_elapsed = time.time() - since
    print(f'\nHuấn luyện hoàn tất trong {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    return model

In [ ]:
def evaluate_model_metrics(model, dataloader):
    """Đánh giá mô hình trên tập test và tính Precision, Recall, F1-macro."""
    print("\nĐang đánh giá chi tiết trên tập Test...")
    model = model.to(device)
    model.eval() # Chuyển sang chế độ eval

    all_labels = []
    all_preds = []

    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Evaluating"):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    precision, recall, f1, _ = precision_recall_fscore_support(
        all_labels,
        all_preds,
        average='macro',
        zero_division=0
    )
    accuracy = accuracy_score(all_labels, all_preds)

    print("\n--- KẾT QUẢ ĐÁNH GIÁ (Macro Average) ---")
    print(f'Accuracy:  {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall:    {recall:.4f}')
    print(f'F1-Score:  {f1:.4f}')
    print("-" * 39)

In [ ]:
# --- Tải Dữ Liệu ---
dataloaders_28, sizes_28, classes_28 = get_dataloaders(img_size=28, batch_size=64)
dataloaders_224, sizes_224, classes_224 = get_dataloaders(img_size=224, batch_size=32)


Đang tải dữ liệu với kích thước 28x28...
Tải xong. 10089 ảnh train, 6693 ảnh test.

Đang tải dữ liệu với kích thước 224x224...
Tải xong. 10089 ảnh train, 6693 ảnh test.


In [ ]:
!pip install evaluate

In [ ]:
# -------------------------------------------------------------------
# BÀI 4: ResNet-50 (HuggingFace)
# -------------------------------------------------------------------
print("\n" + "="*50)
print("BẮT ĐẦU BÀI 4: ResNet-50 (HuggingFace)")
print("="*50)

from datasets import load_dataset
from transformers import AutoImageProcessor, AutoModelForImageClassification, TrainingArguments, Trainer
import evaluate
from huggingface_hub import HfFolder, login

if os.path.exists(os.path.join(DATA_DIR, 'train')):
    # 1. Tải dataset
    print("Đang tải VinaFood21 bằng HuggingFace datasets...")
    dataset = load_dataset("imagefolder", data_dir=DATA_DIR)

    # 2. Lấy nhãn
    labels = dataset["train"].features["label"].names
    label2id, id2label = {}, {}
    for i, label in enumerate(labels):
        label2id[label] = str(i)
        id2label[str(i)] = label
    print(f"Phát hiện {len(labels)} lớp.")

    # 3. Tải Image Processor
    model_checkpoint = "microsoft/resnet-50"
    processor = AutoImageProcessor.from_pretrained(model_checkpoint)

    # 4. Hàm transform
    def transform_hf(examples):
        inputs = processor(examples["image"], return_tensors="pt")
        inputs["label"] = examples["label"]
        return inputs
    dataset = dataset.with_transform(transform_hf)

    # 5. Data collator
    def collate_fn(examples):
        pixel_values = torch.stack([example["pixel_values"] for example in examples])
        labels = torch.tensor([example["label"] for example in examples])
        return {"pixel_values": pixel_values, "label": labels}

    # 6. Tải mô hình
    print("Đang tải mô hình ResNet-50 từ HuggingFace...")
    model_hf = AutoModelForImageClassification.from_pretrained(
        model_checkpoint,
        num_labels=NUM_CLASSES,
        id2label=id2label,
        label2id=label2id,
        ignore_mismatched_sizes=True
    )

    # 7. Hàm tính metrics
    def compute_metrics_hf(eval_pred):
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)
        precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='macro', zero_division=0)
        acc = accuracy_score(labels, predictions)
        return {'accuracy': acc, 'precision_macro': precision, 'recall_macro': recall, 'f1_macro': f1}

    # 8. Training Arguments
    NUM_EPOCHS_B4 = 10 # Số epochs hợp lý cho Fine-tuning

    training_args = TrainingArguments(
        output_dir="./vinafood_resnet50_hf",
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        evaluation_strategy="epoch",
        num_train_epochs=NUM_EPOCHS_B4,
        save_strategy="epoch",
        load_best_model_at_end=True,
        logging_steps=100,
        optim="adamw_torch", # Sử dụng AdamW (biến thể của Adam)
        learning_rate=2e-5,
        remove_unused_columns=False,
        report_to="none",
    )

    # 9. Khởi tạo Trainer
    trainer = Trainer(
        model=model_hf,
        args=training_args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
        tokenizer=processor,
        compute_metrics=compute_metrics_hf,
        data_collator=collate_fn,
    )

    # 10. Huấn luyện
    print(f"\nHuấn luyện ResNet-50 (HuggingFace) trong {NUM_EPOCHS_B4} epochs...")
    trainer.train()

    # 11. Đánh giá
    print("\n[Bài 4: ResNet-50] Đánh giá cuối cùng:")
    eval_results = trainer.evaluate(dataset["test"])

    print("\n--- KẾT QUẢ ĐÁNH GIÁ (HuggingFace) ---")
    print(f"Accuracy:  {eval_results['eval_accuracy']:.4f}")
    print(f"Precision: {eval_results['eval_precision_macro']:.4f}")
    print(f"Recall:    {eval_results['eval_recall_macro']:.4f}")
    print(f"F1-Score:  {eval_results['eval_f1_macro']:.4f}")
    print("-" * 39)

else:
    print("Bỏ qua Bài 4 do không tải được dữ liệu.")